In [ ]:
import json
from pandas import DataFrame, concat, set_option
from sklearn import metrics
import matplotlib.pyplot as plot

set_option('display.max_colwidth', None) 

Load Data

In [ ]:
test_data_file = '../data/sarc/dev-comments/balanced.json'  # DOUBLE CHECK FILE PATH
model_out_file = 'd3_baseline.out'  # CHANGE FILE PATH

with open(test_data_file, 'r') as f:
    test_data = json.load(f)

with open(model_out_file, 'r') as f:
    # save output file in dict
    model_out = {'pred':[], 'gold':[], 'prob_negative':[], 'prob_positive':[]}

    for line in f:
        lst = line.replace(',', '').split()
        # index labels and probabilities from each line, store in dict
        model_out['pred'].append(lst[1])
        model_out['gold'].append(lst[3])
        model_out['prob_negative'].append(lst[5])
        model_out['prob_positive'].append(lst[6])

Create Dataframe

In [ ]:
df1 = DataFrame.from_dict(model_out)
df2 = DataFrame.from_records(test_data)

table = concat([df2['response'], df1], axis=1)

In [ ]:
# quickly check stats
print(table.describe())

Incorrect classifications

In [ ]:
# print 20 of the incorrectly classified examples
print(table.loc[table['pred'] != table['gold']].head(20))

False Positives

In [ ]:
# explore false positives (model label = sarcastic, gold label = not sarcastic)
false_pos = table.loc[(table['pred'] == '1') & (table['gold'] == '0')]
print(false_pos.head(20))
print('count false positives:', false_pos.shape[0])
print('count true positives:', table.loc[(table['pred'] == '1') & (table['gold'] == '1')].shape[0])

False Negatives

In [ ]:
# explore false negatives (model label = not sarcastic, gold label = sarcastic)
false_neg = table.loc[(table['pred'] == '0') & (table['gold'] == '1')]
print(false_neg.head(20))
print('count false negatives:', false_neg.shape[0])
print('count true negatives:', table.loc[(table['pred'] == '0') & (table['gold'] == '0')].shape[0])

Confusion Matrix

In [ ]:
con_mat = metrics.confusion_matrix(model_out['gold'], model_out['pred'])
cm_graphic = metrics.ConfusionMatrixDisplay(con_mat, display_labels=['Not Sarcastic', 'Sarcastic'])
cm_graphic.plot()
plot.show()